In [2]:
%load_ext autoreload
%autoreload 2

from algebra import *
from cache import *
from costs import *
from features import *
from gradients import *
from helpers import *
from model import *
from splits import *

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import csv
import warnings
warnings.filterwarnings('ignore')

In [3]:
SUB_SAMPLE = True
CACHE_DIR = "test/cache/" if SUB_SAMPLE else "cache/"
SUBMISSIONS_DIR = "test/submissions/" if SUB_SAMPLE else "submissions/"

In [4]:
y, x, ids = load_csv_data('data/train.csv', SUB_SAMPLE)

### Logistic Regression

In [5]:
def clean_standardize_expand(y, x, h):
        
    degree = int(h['degree'])

    x = remove_errors(x)
    x = remove_outliers(x)
    x = standardize_all(x)
    x = remove_nan_features(x)
    x = build_poly(x, degree)

    return y, x

def clean_expand(y, x, h):
        
    degree = int(h['degree'])

    x = remove_errors(x)
    x = remove_outliers(x)
    x = remove_nan_features(x)
    x = build_poly(x, degree)

    return y, x

In [6]:
def map_logistic(clean):
    
    def inner_function(y, x, h):
        y, x = clean(y, x, h)
        y = np.where(y == 1, 0, 1)
        return y, x
    
    return inner_function

def logistic_gradient(y, x, w, h):
    
    return compute_logistic_gradient(y, x, w)

def logistic_gradient_ridge(y, x, w, h):
    
    lambda_ = h['lambda']
    
    return compute_logistic_gradient(y, x, w) + lambda_ * w
            
def logistic_error(y, x, w, h):
    
    lambda_ = h['lambda']
    
    logistic_err = compute_logistic_error(y, x, w)
    n_err = compute_error_count(predict_logistic)(y, x, w)
    
    return {
        'logistic_err': logistic_err,
        'n_err': n_err
    }

def logistic_error_and_ridge(y, x, w, h):
    
    lambda_ = h['lambda']
    
    ridge_norm = np.linalg.norm(w, 2) * lambda_
    logistic_err = compute_logistic_error(y, x, w)
    n_err = compute_error_count(predict_logistic)(y, x, w)
    
    return {
        'logistic_err': logistic_err,
        'ridge_norm': ridge_norm,
        'total_loss': logistic_err + ridge_norm,
        'n_err': n_err
    }

In [7]:
def split_data(y, x, ids):
    
    def categorize(x):
        if x[0] == -999.0:
            return 0
        else:
            return x[22] + 1
    
    categories = np.apply_along_axis(categorize, 1, x)

    xs = [x[categories == i] for i in np.arange(5)]
    ys = [y[categories == i] for i in np.arange(5)]
    ids = [ids[categories == i] for i in np.arange(5)]
    
    return ys, xs, ids

In [8]:
y_split, x_split, ids_split = split_data(y, x, ids)

In [9]:
hs = {
    'batch_size': 1,
    'degree': 1,#np.concatenate([[-2], np.arange(1, 4)]),
    'gamma': np.logspace(-3, 0, 8), 
    'k_fold': 4,
    'lambda': np.logspace(-10, -5, 5),
    'max_iters': 5000,
    'num_batches': 1,
    'seed': 1,
    'seed_cv': 1
}

for i in range(1,3):
    cache = Cache(CACHE_DIR + f'A_clean_std_expand_logistic_regression_ridge_cross_validate_split{i}')

    _ = evaluate(
        clean = map_logistic(clean_standardize_expand), 
        fit   = descent_with_cache(
            descent    = cross_validate_descent(
                            stochastic_gradient_descent_e(logistic_gradient_ridge), 
                            logistic_error_and_ridge,
                        ),
            round_size = 100,
            cache      = cache,
            log        = True
        ), 
        x     = x_split[i],
        y     = y_split[i], 
        hs    = hs
    )

iteration 100 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1e-10, 'max_iters': 100, 'num_batches': 1, 'seed': 1, 'seed_cv': 1, 'avg_logistic_err_tr': 0.6700205637812975, 'avg_ridge_norm_tr': 1.9893424315674596e-11, 'avg_total_loss_tr': 0.6700205638011909, 'avg_n_err_tr': 0.423943661971831, 'avg_logistic_err_te': 0.667800810456653, 'avg_ridge_norm_te': 1.9893424315674596e-11, 'avg_total_loss_te': 0.6678008104765464, 'avg_n_err_te': 0.41338028169014085}
iteration 200 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1e-10, 'max_iters': 100, 'num_batches': 1, 'seed': 101, 'seed_cv': 1, 'avg_logistic_err_tr': 0.6408631298557483, 'avg_ridge_norm_tr': 2.3397715114086828e-11, 'avg_total_loss_tr': 0.6408631298791461, 'avg_n_err_tr': 0.3220657276995305, 'avg_logistic_err_te': 0.640186476398752, 'avg_ridge_norm_te': 2.3397715114086828e-11, 'avg_total_loss_te': 0.6401864764221499, 'avg_n_err_te': 0.3154929577464789}
iteration 300 - {'batch_size': 1

iteration 1900 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1e-10, 'max_iters': 100, 'num_batches': 1, 'seed': 1801, 'seed_cv': 1, 'avg_logistic_err_tr': 0.6091944277046305, 'avg_ridge_norm_tr': 7.608916703557808e-11, 'avg_total_loss_tr': 0.6091944277807197, 'avg_n_err_tr': 0.33427230046948353, 'avg_logistic_err_te': 0.5984412047282872, 'avg_ridge_norm_te': 7.608916703557808e-11, 'avg_total_loss_te': 0.5984412048043763, 'avg_n_err_te': 0.3330985915492958}
iteration 2000 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1e-10, 'max_iters': 100, 'num_batches': 1, 'seed': 1901, 'seed_cv': 1, 'avg_logistic_err_tr': 0.6030422901297914, 'avg_ridge_norm_tr': 7.460322495309889e-11, 'avg_total_loss_tr': 0.6030422902043946, 'avg_n_err_tr': 0.3227699530516432, 'avg_logistic_err_te': 0.5918458354792832, 'avg_ridge_norm_te': 7.460322495309889e-11, 'avg_total_loss_te': 0.5918458355538864, 'avg_n_err_te': 0.3190140845070423}
iteration 2100 - {'batch_si

iteration 3600 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1e-10, 'max_iters': 100, 'num_batches': 1, 'seed': 3501, 'seed_cv': 1, 'avg_logistic_err_tr': 0.5681082922560274, 'avg_ridge_norm_tr': 9.333894838358652e-11, 'avg_total_loss_tr': 0.5681082923493663, 'avg_n_err_tr': 0.29295774647887324, 'avg_logistic_err_te': 0.5599830926700569, 'avg_ridge_norm_te': 9.333894838358652e-11, 'avg_total_loss_te': 0.5599830927633959, 'avg_n_err_te': 0.28169014084507044}
iteration 3700 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1e-10, 'max_iters': 100, 'num_batches': 1, 'seed': 3601, 'seed_cv': 1, 'avg_logistic_err_tr': 0.5659161290608961, 'avg_ridge_norm_tr': 9.61608181476497e-11, 'avg_total_loss_tr': 0.5659161291570569, 'avg_n_err_tr': 0.2906103286384977, 'avg_logistic_err_te': 0.5570855561914816, 'avg_ridge_norm_te': 9.61608181476497e-11, 'avg_total_loss_te': 0.5570855562876426, 'avg_n_err_te': 0.28450704225352114}
iteration 3800 - {'batch_si

iteration 400 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1.7782794100389228e-09, 'max_iters': 100, 'num_batches': 1, 'seed': 301, 'seed_cv': 1, 'avg_logistic_err_tr': 0.6618125149204581, 'avg_ridge_norm_tr': 5.504738239963161e-10, 'avg_total_loss_tr': 0.6618125154709319, 'avg_n_err_tr': 0.41032863849765255, 'avg_logistic_err_te': 0.654863632676222, 'avg_ridge_norm_te': 5.504738239963161e-10, 'avg_total_loss_te': 0.6548636332266959, 'avg_n_err_te': 0.38802816901408455}
iteration 500 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1.7782794100389228e-09, 'max_iters': 100, 'num_batches': 1, 'seed': 401, 'seed_cv': 1, 'avg_logistic_err_tr': 0.6462286824479071, 'avg_ridge_norm_tr': 6.53182855172015e-10, 'avg_total_loss_tr': 0.6462286831010899, 'avg_n_err_tr': 0.36596244131455397, 'avg_logistic_err_te': 0.6403164705271075, 'avg_ridge_norm_te': 6.53182855172015e-10, 'avg_total_loss_te': 0.6403164711802903, 'avg_n_err_te': 0.3535211267605634

iteration 2200 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1.7782794100389228e-09, 'max_iters': 100, 'num_batches': 1, 'seed': 2101, 'seed_cv': 1, 'avg_logistic_err_tr': 0.6270119399411793, 'avg_ridge_norm_tr': 1.3755066567392418e-09, 'avg_total_loss_tr': 0.6270119413166859, 'avg_n_err_tr': 0.3471830985915493, 'avg_logistic_err_te': 0.6151442709767593, 'avg_ridge_norm_te': 1.3755066567392418e-09, 'avg_total_loss_te': 0.615144272352266, 'avg_n_err_te': 0.3295774647887324}
iteration 2300 - {'batch_size': 1, 'degree': 1, 'gamma': 0.001, 'k_fold': 4, 'lambda': 1.7782794100389228e-09, 'max_iters': 100, 'num_batches': 1, 'seed': 2201, 'seed_cv': 1, 'avg_logistic_err_tr': 0.6528380626731158, 'avg_ridge_norm_tr': 1.387753055389905e-09, 'avg_total_loss_tr': 0.652838064060869, 'avg_n_err_tr': 0.3744131455399061, 'avg_logistic_err_te': 0.6404153484022125, 'avg_ridge_norm_te': 1.387753055389905e-09, 'avg_total_loss_te': 0.6404153497899656, 'avg_n_err_te': 0.355633802816

...

iteration 900 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 5.62341325190349e-07, 'max_iters': 100, 'num_batches': 1, 'seed': 801, 'seed_cv': 1, 'avg_logistic_err_tr': 2.7686635381720537, 'avg_ridge_norm_tr': 4.331162527730588e-06, 'avg_total_loss_tr': 2.7686678693345814, 'avg_n_err_tr': 0.38572124756335285, 'avg_logistic_err_te': 2.840700205169152, 'avg_ridge_norm_te': 4.331162527730588e-06, 'avg_total_loss_te': 2.84070453633168, 'avg_n_err_te': 0.40789473684210525}
iteration 1000 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 5.62341325190349e-07, 'max_iters': 100, 'num_batches': 1, 'seed': 901, 'seed_cv': 1, 'avg_logistic_err_tr': 2.9279934057496737, 'avg_ridge_norm_tr': 4.447766648579286e-06, 'avg_total_loss_tr': 2.9279978535163225, 'avg_n_err_tr': 0.38523391812865493, 'avg_logistic_err_te': 2.9940639847030925, 'avg_ridge_norm_te': 4.447766648579286e-06, 'avg_total_loss_te': 2.994068432469741, 'avg_n_err_te': 0.4027777777777778}
iterat

iteration 2700 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 5.62341325190349e-07, 'max_iters': 100, 'num_batches': 1, 'seed': 2601, 'seed_cv': 1, 'avg_logistic_err_tr': 4.329769536445273, 'avg_ridge_norm_tr': 7.265048420179167e-06, 'avg_total_loss_tr': 4.3297768014936935, 'avg_n_err_tr': 0.42470760233918126, 'avg_logistic_err_te': 4.342110152648967, 'avg_ridge_norm_te': 7.265048420179167e-06, 'avg_total_loss_te': 4.342117417697387, 'avg_n_err_te': 0.41081871345029236}
iteration 2800 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 5.62341325190349e-07, 'max_iters': 100, 'num_batches': 1, 'seed': 2701, 'seed_cv': 1, 'avg_logistic_err_tr': 4.918590059323189, 'avg_ridge_norm_tr': 7.0628580174865495e-06, 'avg_total_loss_tr': 4.918597122181207, 'avg_n_err_tr': 0.47100389863547765, 'avg_logistic_err_te': 4.970275381596155, 'avg_ridge_norm_te': 7.0628580174865495e-06, 'avg_total_loss_te': 4.970282444454172, 'avg_n_err_te': 0.4700292397660819}
iter

iteration 4500 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 5.62341325190349e-07, 'max_iters': 100, 'num_batches': 1, 'seed': 4401, 'seed_cv': 1, 'avg_logistic_err_tr': 3.2089422008039215, 'avg_ridge_norm_tr': 6.872729560440989e-06, 'avg_total_loss_tr': 3.208949073533482, 'avg_n_err_tr': 0.3983918128654971, 'avg_logistic_err_te': 3.1757198479460476, 'avg_ridge_norm_te': 6.872729560440989e-06, 'avg_total_loss_te': 3.1757267206756077, 'avg_n_err_te': 0.40131578947368424}
iteration 4600 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 5.62341325190349e-07, 'max_iters': 100, 'num_batches': 1, 'seed': 4501, 'seed_cv': 1, 'avg_logistic_err_tr': 3.461199857503867, 'avg_ridge_norm_tr': 6.9371022485172175e-06, 'avg_total_loss_tr': 3.461206794606116, 'avg_n_err_tr': 0.4349415204678363, 'avg_logistic_err_te': 3.4551507347170665, 'avg_ridge_norm_te': 6.9371022485172175e-06, 'avg_total_loss_te': 3.4551576718193155, 'avg_n_err_te': 0.42251461988304095}
i

iteration 1400 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 1e-05, 'max_iters': 100, 'num_batches': 1, 'seed': 1301, 'seed_cv': 1, 'avg_logistic_err_tr': 3.569432219133388, 'avg_ridge_norm_tr': 0.0001113414401913948, 'avg_total_loss_tr': 3.5695435605735786, 'avg_n_err_tr': 0.41642300194931775, 'avg_logistic_err_te': 3.412339688695012, 'avg_ridge_norm_te': 0.0001113414401913948, 'avg_total_loss_te': 3.4124510301352036, 'avg_n_err_te': 0.4078947368421052}
iteration 1500 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 1e-05, 'max_iters': 100, 'num_batches': 1, 'seed': 1401, 'seed_cv': 1, 'avg_logistic_err_tr': 3.689661230090688, 'avg_ridge_norm_tr': 0.00011322685797222658, 'avg_total_loss_tr': 3.68977445694866, 'avg_n_err_tr': 0.4227582846003899, 'avg_logistic_err_te': 3.452413115700236, 'avg_ridge_norm_te': 0.00011322685797222658, 'avg_total_loss_te': 3.4525263425582082, 'avg_n_err_te': 0.38961988304093564}
iteration 1600 - {'batch_size': 1,

iteration 3300 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 1e-05, 'max_iters': 100, 'num_batches': 1, 'seed': 3201, 'seed_cv': 1, 'avg_logistic_err_tr': 4.749714175974326, 'avg_ridge_norm_tr': 0.00012239396358695444, 'avg_total_loss_tr': 4.749836569937912, 'avg_n_err_tr': 0.4722222222222222, 'avg_logistic_err_te': 4.5890663112615355, 'avg_ridge_norm_te': 0.00012239396358695444, 'avg_total_loss_te': 4.589188705225122, 'avg_n_err_te': 0.4627192982456141}
iteration 3400 - {'batch_size': 1, 'degree': 1, 'gamma': 1.0, 'k_fold': 4, 'lambda': 1e-05, 'max_iters': 100, 'num_batches': 1, 'seed': 3301, 'seed_cv': 1, 'avg_logistic_err_tr': 4.508549598582323, 'avg_ridge_norm_tr': 0.0001229163381208359, 'avg_total_loss_tr': 4.508672514920444, 'avg_n_err_tr': 0.4519980506822612, 'avg_logistic_err_te': 4.374988706175344, 'avg_ridge_norm_te': 0.0001229163381208359, 'avg_total_loss_te': 4.375111622513465, 'avg_n_err_te': 0.44298245614035087}
iteration 3500 - {'batch_size': 1, '

In [ ]:
def compute_mle(y, x, w):
    res = 0
    for i in range(x.shape[0]):
        xnTW = x[i].T@w
        res += np.log(1+np.exp(xnTW))-y[i]*xnTW
    return res / y.shape[0]

def print_values(y, x, w):
    for i in range(y.shape[0]):
        print("y: " + str(y[i]))
        print("pred :  " + str(x[i].T@w))

def compute_mle2(y, x, w):
    y_pred = logistic_function(x @ w)
    return - (y @ np.log(y_pred) + (1 - y) @ np.log(1 - y_pred))

class First_Order_Logistic_Regression_Model2(Model):

    def prepare(self, x, y, h):
        
        degree = int(h['degree'])

        x = remove_errors(x)
        x = remove_outliers(x)
        #x = standardize_all(x)
        x = remove_nan_features(x)
        x = build_poly(x, degree)
        #print('avant')
        #print(y[:5])
        y = np.where(y == -1, 0, 1)
        #print('après')
        #print(y[:5])
        
        return x, y

    def fit(self, x, y, h={}):
        
        batch_size = int(h['batch_size'])
        n_iters = int(h['n_iters'])
        gamma = float(h['gamma'])
        
        initial_w = np.zeros(x.shape[1])
        return logistic_regression(y, x, initial_w, batch_size, n_iters, gamma)
    
    def test(self, x, y, w, h):
        #y = np.where(y == 0, -1, 1)
        mse = compute_mle(y, x, w)
        #print_values(y, x, w)
        if np.isnan(mse):
            mse = np.inf
        return { 'mse': mse }

In [ ]:
myModel = CrossValidationModel(First_Order_Logistic_Regression_Model2())

n_iters = [100]
batch_size = [50]

degrees = np.arange(1,4)
gammas = np.logspace(-20, -10, 5)
gammas = gammas[:len(gammas)-1]

hs={
    'n_iters': n_iters,
    'batch_size': batch_size,
    'degree': degrees,
    'gamma': gammas,
    'k_fold': [4],
    'seed': [0]
}

res = myModel.evaluate(x, y, hs, filename=CACHE_DIR+'Logistic_Regression_ExploFullSampleNoStdProp')

#print(res)

plot_heatmap(res, hs, 'mse_te', 'degree', 'gamma')

#res_mse = np.vectorize(lambda x: x['mse'])(res)
#x_axis = np.unique(np.vectorize(lambda x: x['gamma'])(res))
#y_axis = np.unique(np.vectorize(lambda x: x['degree'])(res))

#plot_heatmap(res, hs, 'mse', 'degree', 'gamma')
find_arg_min(res, 'mse_te')

In [ ]:
hs = {'batch_size': 1.0,
 'degree': 6.0,
 'gamma': 10**-20,
 'k_fold': 4.0,
 'n_iters': 10.0,
 'seed': 0.0,
 'mse_te': 0.3988701335328735,
 'mse_tr': 0.3988200000042866}

myModel.predict(hs, x, y, SUBMISSIONS_DIR + 'Logistic_Regression')

### Regularized Order Logistic Regression

In [ ]:
class Regularized_Logistic_Regression_Model(Model):

    def prepare(self, x, y, h):
        
        degree = int(h['degree'])

        x = remove_errors(x)
        x = remove_outliers(x)
        #x = standardize_all(x)
        x = remove_nan_features(x)
        x = build_poly(x, degree)
        
        y = np.where(y == -1, 0, 1)
        
        return x, y

    def fit(self, x, y, h={}):
        
        batch_size = int(h['batch_size'])
        n_iters = int(h['n_iters'])
        gamma = float(h['gamma'])
        lambda_ = float(h['lambda'])
        
        initial_w = np.zeros(x.shape[1])
        return reg_logistic_regression(y, x, initial_w, batch_size, n_iters, gamma, lambda_)
    
    def test(self, x, y, w, h):
        mse = compute_mle(y, x, w)
        if np.isnan(mse):
            mse = np.inf
        return { 'mse': mse }
    
    def predict(self, h, x_tr, y_tr, name):

        x_tr, y_tr = self.prepare(x_tr, y_tr, h)
        w = self.fit(x_tr, y_tr, h)

        _, x_pred, ids = load_csv_data("data/test.csv", sub_sample=False)
        x_pred, _ = self.prepare(x_pred, None, h)
        y_pred = np.dot(data, weights)
        #modified for logistic regression
        y_pred[np.where(y_pred <= 0.5)] = -1
        y_pred[np.where(y_pred > 0.5)] = 1

        create_csv_submission(ids, y_pred, name)
        

In [ ]:
myModel = CrossValidationModel(Regularized_Logistic_Regression_Model())

n_iters = [1000]
batch_size = [1]
degrees = 4#np.arange(1,8)
gammas = 10**-7 #np.logspace(-10, -5, 10)
lambdas = 10**-8#np.logspace(-8, -5, 3)

hs={
    'n_iters': n_iters,
    'batch_size': batch_size,
    'degree': degrees,
    'gamma': gammas,
    'lambda': lambdas,
    'k_fold': [4],
    'seed': [0]
}

res = myModel.evaluate(x, y, hs, filename=CACHE_DIR+'Regularized_Logistic_Regression_Explo')


#plot_heatmap(res, hs, 'mse_te', 'degree', 'gamma')

#res_mse = np.vectorize(lambda x: x['mse'])(res)
#x_axis = np.unique(np.vectorize(lambda x: x['gamma'])(res))
#y_axis = np.unique(np.vectorize(lambda x: x['degree'])(res))

plot_heatmap(res, hs, 'mse_te', 'degree', 'lambda')
plt.figure(2)
plot_heatmap(res, hs, 'mse_te', 'degree', 'gamma')
find_arg_min(res, 'mse_te')

In [ ]:
hs = {'batch_size': 1,
 'degree': 4,
 'gamma': 1e-07,
 'k_fold': 4,
 'lambda': 1e-08,
 'n_iters': 10000,
 'seed': 0,
 'mse_tr': 0.6896604235218645,
 'mse_te': 0.6896430727585345}


myModel.predict(hs, x, y, SUBMISSIONS_DIR + 'Regularized_Logistic_RegressionNoStd')



In [ ]:
import pandas
csv_file = SUBMISSIONS_DIR + 'Regularized_Logistic_RegressionNoStd'

data = pandas.read_csv(csv_file)

l = data.Prediction

counter = 0
for i in l:
    if i == 1:
        counter += 1

print(counter/568238)

### Ridge

In [ ]:
class RidgeRegression_MSE_Degree_Model(Model):

    def prepare(self, x, y, h):
        
        degree = int(h['degree'])

        x = remove_errors(x)
        x = remove_outliers(x)
        x = standardize_all(x)
        x = remove_nan_features(x)
        x = build_poly(x, degree)
        
        return x, y

    def fit(self, x, y, h={}):

        lambda_ = float(h['lambda'])

        return ridge_regression(y, x, lambda_)
    
    def test(self, x, y, w, h):
        mse = compute_mse(y, x, w)
        if np.isnan(mse):
            mse = np.inf
        return { 'mse': mse }


In [ ]:
myModel = CrossValidationModel(RidgeRegression_MSE_Degree_Model())

degrees = np.arange(10, 14)
lambdas = np.logspace(-6, -4,10)

hs={
    'degree': degrees,
    'lambda': lambdas,
    'k_fold': [4],
    'seed': [0]
}

res = myModel.evaluate(x, y, hs, filename=CACHE_DIR+'Ridge_Explo_Vinc')


plot_heatmap(res, hs, 'mse_te', 'degree', 'lambda')
find_arg_min(res, 'mse_te')

### Lasso

In [ ]:
class Lasso_SGD_MSE_Degree_Model(Model):

    def prepare(self, x, y, h):
        
        degree = int(h['degree'])

        x = remove_errors(x)
        x = remove_outliers(x)
        x = standardize_all(x)
        x = remove_nan_features(x)
        x = build_poly(x, degree)
        
        return x, y

    def fit(self, x, y, h={}):
        
        batch_size = int(h['batch_size'])
        n_iters = int(h['n_iters'])
        lambda_ = float(h['lambda'])
        gamma = float(h['gamma'])

        initial_w = np.zeros(x.shape[1])

        return lasso_stochastic_gradient_descent(y, x, initial_w, batch_size, n_iters, gamma, lambda_)
    
    def test(self, x, y, w, h):
            mse = compute_mse(y, x, w)
            if np.isnan(mse):
                mse = np.inf
            return { 'mse': mse }

In [ ]:
myModel = CrossValidationModel(Lasso_SGD_MSE_Degree_Model())

degrees = np.arange(4)
lambdas = np.logspace(-3, -1,5)
gammas = np.logspace(-15, -10, 3)

hs={
    'batch_size': 1,
    'n_iters': 1000,
    'degree': degrees,
    'lambda': lambdas,
    'gamma': gammas,
    'k_fold': [4],
    'seed': [0]
}

res = myModel.evaluate(x, y, hs, filename=CACHE_DIR+'Lasso_Explo_VincTEST')


plot_heatmap(res, hs, 'mse_te', 'degree', 'lambda')
plot_heatmap(res, hs, 'mse_te', 'degree', 'gamma')
find_arg_min(res, 'mse_te')